### Progetto 8b
Luca Maria Lauricella 519997

Valerio Marini 512489

Repository del progetto:
https://github.com/lauriluca99/TGW-3D.jl

Documentazione del progetto:
https://lauriluca99.github.io/TGW-3D.jl

In [ ]:
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation
using LinearAlgebra
Lar=LinearAlgebraicRepresentation

    function spatial_arrangement_2(
        rV::Points, 
        rcopEV::ChainOp, 
        rcopFE::ChainOp, 
        [multiproc::Bool=false])
			
Effettua la ricostruzione delle facce permettendo il wrapping spaziale 3D.
#### Argomenti addizionali:
- `multiproc::Bool`: Esegue la computazione in modalità parallela. Di Default a `false`.

In [ ]:
function spatial_arrangement_2(
    rV::Lar.Points,
    rcopEV::Lar.ChainOp,
    rcopFE::Lar.ChainOp, 
    multiproc::Bool=false)

#rcopCF = Lar.build_copFC(rV, rcopEV, rcopFE)  ######
rcopCF = Lar.Arrangement.minimal_3cycles(rV, rcopEV, rcopFE)
return rV, rcopEV, rcopFE, rcopCF
end

In [ ]:
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])

In [4]:
@benchmark spatial_arrangement_2(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

In [ ]:
@code_warntype spatial_arrangement_2(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

# Ottimizzazione

In [ ]:
function minimal_3cycles(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp)

	triangulated_faces = Array{Any, 1}(undef, FE.m)

    function face_angle(e::Int, f::Int)
        if !isassigned(triangulated_faces, f)
            @views vs_idxs = Array{Int64, 1}()
            edges_idxs = FE[f, :].nzind
            edge_num = length(edges_idxs)
            edges = zeros(Int64, edge_num, 2)

            for (i, ee) in enumerate(edges_idxs)
                edge = EV[ee, :].nzind
                edges[i, :] = edge
                vs_idxs = union(vs_idxs, edge)
            end

            #vs = V[vs_idxs, :]
			fv,edges = Lar.vcycle(EV, FE, f)

			vs = V[fv, :]


            v1 = LinearAlgebra.normalize(vs[2, :] - vs[1, :])
            v2 = [0 0 0]		# added for debug
            v3 = [0 0 0]
            err = 1e-8
            i = 3
            while -err < norm(v3) < err
                v2 = normalize(vs[i, :] - vs[1, :])
                v3 = cross(v1, v2)
                i = i + 1
            end
           M = reshape([v1; v2; v3], 3, 3)

            #vs = vs*M
			vs = (vs*M)[:, 1:2]

            # triangulated_faces[f] = Triangle.constrained_triangulation(
            #     Array{Float64,2}(vs), vs_idxs, edges, fill(true, edge_num))
			v = convert(Lar.Points, vs'[1:2,:])
			vmap = Dict(zip(fv,1:length(fv))) # vertex map
			mapv = Dict(zip(1:length(fv),fv)) # inverse vertex map
			trias = Lar.triangulate2d(v,edges)
			@views triangulated_faces[f] = [[mapv[v] for v in tria] for tria in trias]
        end
        edge_vs = EV[e, :].nzind

        t = findfirst(x->edge_vs[1] in x && edge_vs[2] in x, triangulated_faces[f])

        v1 = normalize(V[edge_vs[2], :] - V[edge_vs[1], :])

        if abs(v1[1]) > abs(v1[2])
            invlen = 1. / sqrt(v1[1]*v1[1] + v1[3]*v1[3])
            v2 = [-v1[3]*invlen, 0, v1[1]*invlen]
        else
            invlen = 1. / sqrt(v1[2]*v1[2] + v1[3]*v1[3])
            v2 = [0, -v1[3]*invlen, v1[2]*invlen]
        end

        v3 = cross(v1, v2)

        M = reshape([v1; v2; v3], 3, 3)

        triangle = triangulated_faces[f][t]
        third_v = setdiff(triangle, edge_vs)[1]
        vs = V[[edge_vs..., third_v], :]*M

        v = vs[3, :] - vs[1, :]
        angle = atan(v[2], v[3])
        return angle
    end

    #EF = FE'
    EF::SparseArrays.SparseMatrixCSC{Int8,Int} = convert(Lar.ChainOp, LinearAlgebra.transpose(FE))
	FC::SparseMatrixCSC{Int64, Int64} = Lar.Arrangement.minimal_cycles(face_angle, true)(V, EF)

	#FC'
    return -convert(Lar.ChainOp, LinearAlgebra.transpose(FC))
end



In [ ]:
@code_warntype minimal_3cycles(V, EV, FE)

In [ ]:
function spatial_arrangement_21(
    rV::Lar.Points,
    rcopEV::Lar.ChainOp,
    rcopFE::Lar.ChainOp, 
    multiproc::Bool=false)

#rcopCF = Lar.build_copFC(rV, rcopEV, rcopFE)  ######
rcopCF::SparseArrays.SparseMatrixCSC{Int8,Int} = minimal_3cycles(rV, rcopEV, rcopFE)
return rV, rcopEV, rcopFE, rcopCF
end

In [ ]:
@code_warntype spatial_arrangement_21(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

In [ ]:
@benchmark spatial_arrangement_21(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)